In [2]:
# test_phase_4b_sweep_corrected.ipynb

import numpy as np
import pandas as pd
from neural_mi.data.processors import ContinuousProcessor

# Important: We need to import the class from the file we just created
from neural_mi.analysis.sweep import ParameterSweep 

if __name__ == '__main__':
    print(f"--- Testing Phase 4b: The ParameterSweep Engine (Corrected) ---")
    device = 'cpu'

    # --- 1. Generate Data with DIFFERENT Dimensions ---
    N_SAMPLES = 8000
    # X has 5 channels, Y has 2 channels
    x_raw = np.random.randn(5, N_SAMPLES) 
    y_raw = np.random.randn(2, N_SAMPLES)
    # Add correlation
    y_raw[0, :] += x_raw[0, :] 
    
    cont_proc_x = ContinuousProcessor(window_size=100, step_size=50)
    x_data = cont_proc_x.process(x_raw).to(device)
    
    # Use a different window size for Y for a more robust test
    cont_proc_y = ContinuousProcessor(window_size=100, step_size=50)
    y_data = cont_proc_y.process(y_raw).to(device)
    
    print(f"Data shapes: X={x_data.shape}, Y={y_data.shape}")
    print("-" * 40)

    # --- 2. Define Parameters for the Sweep ---
    base_params = {
        'hidden_dim': 128, 'n_layers': 2, 'learning_rate': 1e-4,
        'n_epochs': 5, 'batch_size': 128, 'patience': 3
    }
    sweep_grid = {'embedding_dim': [8, 16, 32]}

    # --- 3. Instantiate and Run the Sweep ---
    sweep = ParameterSweep(x_data, y_data, base_params, critic_type='separable')
    results = sweep.run(sweep_grid=sweep_grid, n_workers=2)

    # --- 4. Verification ---
    print("\n--- Verification ---")
    assert len(results) == len(sweep_grid['embedding_dim'])
    print(f"Successfully collected results from {len(results)} sweep tasks.")

    results_df = pd.DataFrame(results)
    print("\nParameter Sweep Results Summary:")
    print(results_df[['embedding_dim', 'best_epoch', 'train_mi', 'test_mi']].round(4))
    
    print("\n✅ Phase 4b (Sweep) test completed successfully!")

--- Testing Phase 4b: The ParameterSweep Engine (Corrected) ---
Data shapes: X=torch.Size([159, 5, 100]), Y=torch.Size([159, 2, 100])
----------------------------------------
Starting parameter sweep with 2 workers...
Created 3 tasks for the sweep...
Epoch 1/5 | Test MI: -0.0279
Epoch 2/5 | Test MI: -0.0277
Epoch 3/5 | Test MI: -0.0275
Epoch 4/5 | Test MI: -0.0274
Epoch 5/5 | Test MI: -0.0272
Best epoch identified (via smoothed curve): 5 (Smoothed MI: -0.0273)
Epoch 1/5 | Test MI: 0.0321
Epoch 2/5 | Test MI: 0.0324
Epoch 3/5 | Test MI: 0.0329
Epoch 4/5 | Test MI: 0.0334
Epoch 5/5 | Test MI: 0.0340
Best epoch identified (via smoothed curve): 5 (Smoothed MI: 0.0338)
Epoch 1/5 | Test MI: -0.0076
Epoch 2/5 | Test MI: -0.0076
Epoch 3/5 | Test MI: -0.0076
Epoch 4/5 | Test MI: -0.0075
Epoch 5/5 | Test MI: -0.0074
Best epoch identified (via smoothed curve): 5 (Smoothed MI: -0.0075)
Parameter sweep finished.

--- Verification ---
Successfully collected results from 3 sweep tasks.

Parameter Swe

In [2]:
results_df

,hidden_dim,n_layers,learning_rate,n_epochs,batch_size,patience,critic_type,estimator_fn,use_variational,input_dim,embedding_dim,train_mi,test_mi,best_epoch,test_mi_history
0,128,2,0.0001,5,128,3,separable,<function infonce_lower_bound at 0x306ad79c0>,False,500,8,0.105156,-0.068263,4,"[-0.07385444641113281, -0.07214570045471191, -..."
1,128,2,0.0001,5,128,3,separable,<function infonce_lower_bound at 0x306ad79c0>,False,500,16,0.080801,-0.013345,4,"[-0.015492439270019531, -0.014634847640991211,..."
2,128,2,0.0001,5,128,3,separable,<function infonce_lower_bound at 0x306ad79c0>,False,500,32,0.070618,-0.042864,4,"[-0.04577136039733887, -0.0448756217956543, -0..."
